# Decision tree

In [ ]:
from sklearn import tree

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
import graphviz

## Visualize the Decision Boundary

In [ ]:
import numpy as np, seaborn as sns, matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
def gini(p):
    return 2*(p)*(1-p)
def entropy(p):
    return - p*np.log2(p) - (1-p)*np.log2((1-p))
def error(p):
    return 1 - np.max([p,1-p])

## Modelling End-to-End with Decision Tree

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.metrics import accuracy_score

## Project HR

Download sample HR data from https://www.ibm.com/communities/analytics/watson-analytics-blog/hr-employee-attrition/ WA_Fn-UseC_-HR-Employee-Attrition.xlsx and save as csv.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.pop('EmployeeCount')
df.pop('EmployeeNumber')
df.pop('Over18')
df.pop('StandardHours')

In [ ]:
df.columns

In [ ]:
y=df['Attrition']
X = df
X.pop('Attrition')

In [ ]:
y.unique()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelBinarizer()


In [ ]:
y=le.fit_transform(y)

In [ ]:
y

In [ ]:
y.shape

In [ ]:
df.info()

In [ ]:
df.select_dtypes(['object'])

In [ ]:
ind_BusinessTravel = pd.get_dummies(df['BusinessTravel'],prefix='BusinessTravel')
ind_Department = pd.get_dummies(df['Department'],prefix='Department')
ind_EducationField = pd.get_dummies(df['EducationField'],prefix='EducationField')
ind_Gender = pd.get_dummies(df['Gender'],prefix='Gender')
ind_JobRole = pd.get_dummies(df['JobRole'],prefix='JobRole')
ind_MaritalStatus = pd.get_dummies(df['MaritalStatus'],prefix='MaritalStatus')
ind_OverTime = pd.get_dummies(df['OverTime'],prefix='OverTime')

In [ ]:
df1 =pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, ind_JobRole,
                ind_MaritalStatus, ind_OverTime, df.select_dtypes(['int64'])]
               , axis=1)

In [ ]:
df1

In [ ]:
df1.shape

### Decision Tree

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df1,y)

In [ ]:
dtc = tree.DecisionTreeClassifier(random_state=42)

In [ ]:
dtc = dtc.fit(X_train, y_train)

In [ ]:
dtc

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy_score(y_train,dtc.predict(X_train))

In [ ]:
print(classification_report(y_train,dtc.predict(X_train)))

In [ ]:
confusion_matrix(y_train,dtc.predict(X_train))

In [ ]:
print(classification_report(y_test,dtc.predict(X_test)))

In [ ]:
confusion_matrix(y_test,dtc.predict(X_test))

In [ ]:
def print_training_score(clf, X, y):
    print("Training Result:\n")
    print("Accuracy: {0:.4f}\n".format(accuracy_score(y, clf.predict(X))))
    print("Classification Report: \n {} \n".format(classification_report(y, clf.predict(X))))
    print("Confusion Matrix: \n {} \n".format(confusion_matrix(y, clf.predict(X))))

    res = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
    print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))

In [ ]:
def print_test_score(clf, X, y):
    print("Test Result:\n")

    print("Accuracy: {0:.4f}\n".format(accuracy_score(y, clf.predict(X))))
    print("Classification Report: \n {} \n".format(classification_report(y, clf.predict(X))))
    print("Confusion Matrix: \n {} \n".format(confusion_matrix(y, clf.predict(X))))

In [ ]:
print_training_score(dtc, X_train, y_train)

In [ ]:
print_test_score(dtc, X_test, y_test)

## Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag_clf = BaggingClassifier(base_estimator=dtc, n_estimators=5000, bootstrap=True, n_jobs=-1, random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train)

In [ ]:
print_training_score(bag_clf, X_train, y_train)
print_test_score(bag_clf, X_test, y_test)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier()

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
print_training_score(rf_clf, X_train, y_train)
print_test_score(rf_clf, X_test, y_test)

In [ ]:
import seaborn as sns

In [ ]:
pd.Series(rf_clf.feature_importances_,
         index=X_train.columns).sort_values(ascending=False).plot(kind='bar', figsize=(12,6));

# Ada Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada_clf = AdaBoostClassifier()

In [ ]:
ada_clf.fit(X_train, y_train)

In [ ]:
print_training_score(ada_clf, X_train, y_train)
print_test_score(ada_clf, X_test, y_test)

# Ada + RandomForest

In [ ]:
arf_clf = AdaBoostClassifier(RandomForestClassifier())

In [ ]:
arf_clf.fit(X_train, y_train)
print_training_score(arf_clf, X_train, y_train)
print_test_score(arf_clf, X_test, y_test)

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc_clf = GradientBoostingClassifier()
gbc_clf.fit(X_train, y_train)
print_training_score(gbc_clf, X_train, y_train)
print_test_score(gbc_clf, X_test, y_test)

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xg_clf = xgb.XGBClassifier()
xg_clf.fit(X_train, y_train)
print_training_score(xg_clf, X_train, y_train)
print_test_score(xg_clf, X_test, y_test)

In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelBinarizer()

In [ ]:
tmp=le.fit_transform(y)

In [ ]:
type(tmp)

In [ ]:
tmp = pd.Series(list(tmp))

In [ ]:
tmp.value_counts()

In [ ]:
tmp.value_counts() / tmp.count()

In [ ]:
df.info()